In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import pandas as pd
import datetime
weather_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/weather.csv')
trip_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/trip.csv')
status_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/status.csv')
station_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/station.csv')
status_trip_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/status_trip.csv')


def strptime_date(df,pattern='%Y-%m-%d %H:%M:%S'):
    return [datetime.datetime.strptime(df[i], pattern) for i in range(len(df))]
    
weather_df["date"]=strptime_date(weather_df["date"],pattern='%Y-%m-%d')
status_trip_df["date"]=strptime_date(status_trip_df["date"])

In [ ]:
#status_dfにdatetimeのdate列を追加
status_df["date"]=[datetime.datetime(status_df["year"][i],status_df["month"][i],status_df["day"][i],status_df["hour"][i]) for i in range(len(status_df))]

#start_date,end_dateをdatetime化
def strptime_trip_date(df):
    return [datetime.datetime.strptime(df[i], '%m/%d/%Y %H:%M') for i in range(len(df))]

trip_df["start_datetime"]=strptime_trip_date(trip_df['start_date'])
trip_df["end_datetime"]=strptime_trip_date(trip_df['end_date'])

#start_datetime,end_datetimeのhourより下を切り捨て
def make_date_key(df):
    return [datetime.datetime(df[i].year,df[i].month,df[i].day,df[i].hour) for i in range(len(df))]

trip_df["start_date_key"]=make_date_key(trip_df["start_datetime"])
trip_df["end_date_key"]=make_date_key(trip_df["end_datetime"])

#status_dfとtrip_dfをmarge
rent_num=trip_df[["start_date_key","start_station_id","start_date"]].groupby(["start_date_key","start_station_id"]).count()
status_trip_df=pd.merge(status_df,rent_num, left_on=['date','station_id'], right_on=['start_date_key',"start_station_id"],how="outer")
return_num=trip_df[["end_date_key","end_station_id","end_date"]].groupby(["end_date_key","end_station_id"]).count()
status_trip_df=pd.merge(status_trip_df,return_num, left_on=['date','station_id'], right_on=['end_date_key',"end_station_id"],how="outer")
status_trip_df.rename(columns={'start_date': 'rent_num', 'end_date': 'return_num'},inplace=True)
status_trip_df.fillna({'rent_num':0,'return_num':0,"bikes_available":0},inplace=True)
status_trip_df=status_trip_df.dropna(how="any")
status_trip_df["diff"]=status_trip_df["return_num"]-status_trip_df["rent_num"]
status_trip_df

In [83]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
def pca_weather(df):
    X=df.drop(["events","date"],axis=1)
    X = StandardScaler().fit_transform(X)
    pca = PCA(n_components=2)
    pca.fit(X)
    #print(pca.explained_variance_ratio_)
    return pd.DataFrame(pca.transform(X),columns=['first component', 'second component'])

pca_weather_df=pca_weather(weather_df)
#pca_weather_df

#status_trip_dfとpca_weather_dfを結合
for i in range(len(weather_df)):
    bool_index=(status_trip_df["date"]<(weather_df.loc[i,"date"]+datetime.timedelta(days=1)))&(status_trip_df["date"]>=weather_df.loc[i,"date"])
    status_trip_df.loc[bool_index,"weather1"]=pca_weather_df.loc[i,'first component']
    status_trip_df.loc[bool_index,"weather2"]=pca_weather_df.loc[i,'second component']

#station_dfと結合
status_trip_df=pd.merge(status_trip_df,station_df, left_on=['station_id'], right_on=['station_id'],how="inner")
status_trip_df.drop("Unnamed: 0",axis=1,inplace=True)
status_trip_df.drop("Unnamed: 0.1",axis=1,inplace=True)

[0.35049866 0.20372581]


In [ ]:
status_trip_df["installation_date"]=strptime_date(status_trip_df["installation_date"],"%m/%d/%Y")
status_trip_df["is_install"]=status_trip_df['installation_date']<=status_trip_df['date']

In [ ]:
import pandas as pd
import datetime

def strptime_date(df,pattern='%Y-%m-%d %H:%M:%S'):
    return [datetime.datetime.strptime(df[i], pattern) for i in range(len(df))]

marged_df=pd.read_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/marged_df.csv')
marged_df["date"]=strptime_date(marged_df["date"])
marged_df["installation_date"]=strptime_date(marged_df["installation_date"])

In [110]:
marged_df.head()

,id,year,month,day,hour,station_id,bikes_available,predict,date,rent_num,return_num,weather1,weather2,lat,long,dock_count,city,installation_date,is_install
0,0.0,2013.0,9.0,1.0,0.0,0.0,11.0,0.0,2013-09-01 00:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
1,1.0,2013.0,9.0,1.0,1.0,0.0,11.0,0.0,2013-09-01 01:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
2,2.0,2013.0,9.0,1.0,2.0,0.0,11.0,0.0,2013-09-01 02:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
3,3.0,2013.0,9.0,1.0,3.0,0.0,11.0,0.0,2013-09-01 03:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True
4,4.0,2013.0,9.0,1.0,4.0,0.0,11.0,0.0,2013-09-01 04:00:00,0.0,0.0,-2.753002,-1.330403,37.32973,-121.90178,27,city1,2013-08-06,True


In [111]:
marged_df.to_csv('/content/drive/MyDrive/signate/シェアサイクルの利用予測/marged_df.csv',index=False)